In [1]:
import re
import torch
import pandas as pd

from transformers import pipeline
from googletrans import Translator
from transformers import AutoTokenizer, AutoModelForTokenClassification

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
# model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

# nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# device = 0 if torch.cuda.is_available() else -1
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing 

In [3]:
df = pd.read_csv("../dataset/cleaned_train.csv")
translator = Translator()
translator.raise_Exception = True

In [4]:
def replace_person_name(dialogue):
    lines = dialogue.split('\n')
    for i, line in enumerate(lines):
        sp_line = line.split(': ')
        if len(sp_line) == 2:
            speaker, sentence = sp_line
            translated = translator.translate(sentence, src='ko', dest='en')
            ner_result = nlp(translated.text)
            
            # NER 결과에서 사람이름(Person)이고 확률이 45% 이상인 경우 치환
            for entity in ner_result:
                if entity['entity_group'] == 'PER' and entity['score'] >= 0.45:
                    # 원문 한국어 문장에서 해당 이름을 치환
                    name = entity['word']
                    translated_name = translator.translate(name, src='en', dest='ko').text
                    sentence = re.sub(re.escape(translated_name), "#PersonName#", sentence)

            # 다시 한국어로 번역하여 치환된 문장으로 대체
            re_translated = translator.translate(sentence, src='en', dest='ko')
            lines[i] = f"{speaker}: {re_translated.text}"
    
    return '\n'.join(lines)

In [5]:
df['dialogue'] = df['dialogue'].apply(replace_person_name)
print(df.iloc[0]['dialogue'])

Exception: Unexpected status code "429" from ('translate.google.com',)